# Data Ingestion

In [1]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_path='VIIT_ExamRegulations.pdf')
documents = loader.load()
len(documents)

51

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)

docs = text_splitter.split_documents(documents)

len(docs)

415

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\bhanu\OneDrive\navigator\envs\final_project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [5]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(docs))]

vector_store.add_documents(documents=docs, ids=uuids)

['af3f1283-7cc2-476f-9a21-86c3c976ccb0',
 'f4388f06-8d0f-40ec-8e7a-0e91ae13b063',
 '512d7b0c-e927-495d-b2b4-9ad60cad01c5',
 'fc92913d-7454-4cbc-89d4-143f924d09c4',
 'c6ae0eb8-fd70-421a-b148-576f01ff1e04',
 '2a032d5a-c33d-4bde-a4a3-f09e1bc37d3d',
 '727ad764-6f1b-442e-b548-07933e7227e5',
 'b2628628-ee0d-4ec7-bf31-d47ed2c5dd2d',
 '7a8ac832-f1c7-40da-bc4e-a49fb7dd972c',
 '9787bd53-f3e7-4c72-a92d-7e9aaaf2fdde',
 '667573af-a864-4954-b6ef-acef45ef13c1',
 '13db95f4-4de8-409f-af83-857545dc1a2d',
 'e50673af-8811-479d-ae67-75a4ecefce19',
 'd8298ff5-40b3-4bed-b056-b2a8f32acede',
 'afd85871-d055-4765-9d64-263557879c3f',
 'd70b441d-833a-4418-8c34-781f00e3a252',
 'e138e654-78ec-45a0-9b5f-3461934d8007',
 'cd486b0c-a210-4e32-aa1d-4ebf783cde30',
 '16c4bf96-8830-4931-98eb-61a676604864',
 'd19f83d5-95a5-4375-8ce5-43974c0cba31',
 '5207d68c-0f2b-4c8a-a12a-1a7caf0ba0bf',
 '0b6a3fe5-c72f-4d79-acd0-a0116ae3166f',
 'b8de461b-a418-4dc9-8aa0-feb9154c4d2e',
 '5ce63b9c-2e81-4c3f-a83f-6d1c9ad48d53',
 'ce5bc39c-cdc3-

# Retrieval

In [10]:
retriever = vector_store.as_retriever(search_type="similarity",
                                                 search_kwargs={"k": 5}
                                                )

In [17]:
source_docs=retriever.invoke("what is this document about?")
source_docs

[Document(id='63052116-5f62-42a4-b3b1-d15869fb5ced', metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2023-02-10T12:09:48+05:30', 'author': 'ADMIN', 'moddate': '2023-02-10T12:09:48+05:30', 'source': 'VIIT_ExamRegulations.pdf', 'total_pages': 51, 'page': 31, 'page_label': '32'}, page_content='Announcements -  during the commencement of examination, before the \ndistribution of question paper \n \n1. Students are advised to fill all the details on title page of main answer book.  \n2. Students are advised to remove any written or printed material on their'),
 Document(id='c6d17b84-9930-464e-a848-62cde993da75', metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2023-02-10T12:09:48+05:30', 'author': 'ADMIN', 'moddate': '2023-02-10T12:09:48+05:30', 'source': 'VIIT_ExamRegulations.pdf', 'total_pages': 51, 'page': 5, 'page_label': '6'}, page_content='identifying the areas of improvement, variance a

In [18]:
content = ''
for i in source_docs:
    content = content + i.page_content
print(content)

Announcements -  during the commencement of examination, before the 
distribution of question paper 
 
1. Students are advised to fill all the details on title page of main answer book.  
2. Students are advised to remove any written or printed material on theiridentifying the areas of improvement, variance analysis etc., 
vii. Empowered to get the question papers and answer sheets printed 
once they are finalized. 
viii. To order for printing of all the stationary required for examination 
branch etc., after the approval of Principal.(Note: Paste your scanned sign at signature place –need not to make PDF) 
NOTE: The template of the question papers is password protected. Please check your 
mobile for the SMS of the password. 
For any clarification contact the Dean Evaluations on mobile: 9550293989over the last 4 years, to meet the overall objectives of the academic program while 
maintaining the transparency and confidentiality needed for a foolproof 
examination section.  
This manual

# Summarization

In [13]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "t5-small"  # or "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

inputs = tokenizer("summarize: " + content, return_tensors="pt", max_length=512, truncation=True)
summary_ids = model.generate(**inputs, max_length=150, min_length=50, length_penalty=2.0)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print(summary)

c:\Users\bhanu\OneDrive\navigator\envs\final_project\Lib\site-packages\transformers\generation\configuration_utils.py:696: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `2.0` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(


invigilators must go to invigilation in time. students must enter examination blocks only with permitted material. iv) students must enter into exam halls only with permitted material and in time. ANNEXURE – 11 GUIDELINES TO LAB OBSERVERS31 ANNEXURE – 2 GUIDELINES FOR INVIGILATOR General.


'to bring awareness among invigilators about the procedure to perform invigilation \nDetailed Guidelines :  \n1. Report at the examination hall at least 30 minutes before the time of  \ncommencement of Examination. Collect the seating arrangement,3 \n \n \nNo. Title Page Nos. \nAnnexures (Detailed Guidelines) \n1 Norms to be followed by students 29 \n2 Guidelines for Invigilator 32 \n3 Guidelines for Squad Members 36 \n4 Guidelines for question paper setter 37 \n5 Guidelines for Evaluator 38 \n6 Guidelines for Chief Evaluator 40iii) Invigilators have to go to invigilation in time. \niv) Students entry into examination blocks are to be monitored  to ensure \nthat students will enter into exam halls only with permitted material \nand in time.  Late permission should not be  granted under normal \nconditions.11) Complete the  details  in external  examiner  feedback  form .  Enclose your  \nsigned  guidelines sheet with feedback form and seal them in an envelope \nand handover them to int

In [19]:
from ctransformers import AutoModelForCausalLM

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    "TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
    model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    model_type="mistral",  # Set to 'llama' if 'mistral' is not supported
    gpu_layers=0  # Set to >0 if GPU acceleration is available
)

question="what is this document about?"
# Define your prompt
prompt = """
Answer the user question based on the context given and not prior knowledge.
------------------
context: {context}
------------------
question: {question}
"""
prompt=prompt.format(context=content, question=question)
# Generate a response
response = model(prompt, max_new_tokens=200)

# Print the response
print(response)


Fetching 1 files: 100%|██████████| 1/1 [00:00<?, ?it/s]


: 